In [ ]:
import pandas as pd
import numpy as np

from db import engine

In [2]:
class Etl():

    def __init__(self):
        
        self.geolocation = None
        self.customers = None
        self.sellers = None
        self.orders = None
        self.order_reviews = None
        self.order_payments = None
        self.product_category = None
        self.products = None
        self.order_items = None
        self.marketing_qualified_leads = None
        self.close_deals = None
    

    def extract(self):
        
        #self.geolocation = pd.read_csv('https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_geolocation_dataset.csv')
        self.customers = pd.read_csv('https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_customers_dataset.csv')
        self.sellers = pd.read_csv('https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_sellers_dataset.csv')
        self.orders = pd.read_csv('https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_orders_dataset.csv')
        self.order_reviews = pd.read_csv('https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_order_reviews_dataset.csv')
        self.order_payments = pd.read_csv('https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_order_payments_dataset.csv')
        self.product_category = pd.read_csv('https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/product_category_name_translation.csv')
        self.products = pd.read_csv('https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_products_dataset.csv')
        self.order_items = pd.read_csv('https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_order_items_dataset.csv')
        self.marketing_qualified_leads = pd.read_csv('datasets\olist_marketing_qualified_leads_dataset.csv')
        self.close_deals = pd.read_csv('datasets\olist_closed_deals_dataset.csv')


    def transform(self, add_product_category=False, validate=False):

        self.customers.rename(columns={'customer_zip_code_prefix': 'zip_code_prefix', 'customer_city': 'city', 'customer_state': 'state'}, inplace=True)
        self.sellers.rename(columns={'seller_zip_code_prefix': 'zip_code_prefix', 'seller_city': 'city', 'seller_state': 'state'}, inplace=True)

        self.geolocation = pd.concat([self.customers[['zip_code_prefix', 'city', 'state']], self.sellers[['zip_code_prefix', 'city', 'state']]], ignore_index=True) 
        self.geolocation.drop_duplicates(inplace=True)

        self.products['product_category_name'].replace({np.nan: 'no_data'}, inplace=True)
        self.products.fillna(0, inplace=True)
        self.products = self.products.astype({'product_name_lenght': 'int64', 'product_description_lenght': 'int64', 'product_photos_qty': 'int64',
            'product_weight_g': 'int64', 'product_length_cm': 'int64', 'product_height_cm': 'int64', 'product_width_cm': 'int64'})

        if validate:
            geolocationDB = pd.read_sql_table('geolocation', engine.connect())
            self.geolocation = geolocationDB.merge(self.geolocation, how='right', on=['zip_code_prefix','city', 'state'])
            self.geolocation = self.geolocation[self.geolocation['geolocation_id'].isnull()]
            self.geolocation.drop('geolocation_id', axis=1, inplace=True)
            self.geolocation.reset_index(drop=True, inplace=True)
            self.geolocation.index += (len(geolocationDB) + 1)
            self.geolocation = self.geolocation.rename_axis('geolocation_id').reset_index()
            geolocationDB = pd.concat([geolocationDB, self.geolocation], ignore_index=True)

            self.customers = self.customers.merge(geolocationDB, how='left', on=['zip_code_prefix','city', 'state'])

            self.sellers = self.sellers.merge(geolocationDB, how='left', on=['zip_code_prefix','city', 'state'])

            if add_product_category and type(add_product_category) == dict:
                product_categoryDB = pd.read_sql_table('product_category', engine.connect())
                self.product_category = pd.DataFrame(add_product_category)
                self.product_category.index += (len(product_categoryDB) + 1)
                self.product_category = self.product_category.rename_axis('product_category_id').reset_index()
                product_categoryDB = pd.concat([product_categoryDB, self.product_category], ignore_index=True)

            self.products = self.products.merge(product_categoryDB, how='left', on=['product_category_name'])

        else:
            self.geolocation.reset_index(drop=True, inplace=True)
            self.geolocation.index += 1
            self.geolocation = self.geolocation.rename_axis('geolocation_id').reset_index()

            self.customers = self.customers.merge(self.geolocation, how='left', on=['zip_code_prefix','city', 'state'])

            self.sellers = self.sellers.merge(self.geolocation, how='left', on=['zip_code_prefix','city', 'state'])

            if add_product_category and type(add_product_category) == dict:
                add_product_category = pd.DataFrame(add_product_category)
                self.product_category = pd.concat([self.product_category, add_product_category], ignore_index=True)
            self.product_category.index += 1
            self.product_category = self.product_category.rename_axis('product_category_id').reset_index()

            self.products = self.products.merge(self.product_category, how='left', on=['product_category_name'])

        
        self.customers.drop(['customer_unique_id', 'zip_code_prefix', 'city', 'state'], axis=1, inplace=True)
        
        self.sellers.drop(['zip_code_prefix', 'city', 'state'], axis=1, inplace=True)
   
        self.orders.fillna('2016-01-01 00:00:00', inplace=True)
        self.orders = self.orders.astype({'order_purchase_timestamp': 'datetime64', 'order_approved_at': 'datetime64', 'order_delivered_carrier_date': 'datetime64',
            'order_delivered_customer_date': 'datetime64', 'order_estimated_delivery_date': 'datetime64'})

        self.order_reviews.drop(['review_comment_title', 'review_comment_message'], axis=1, inplace=True)
        self.order_reviews = self.order_reviews.astype({'review_creation_date': 'datetime64', 'review_answer_timestamp': 'datetime64'})
        self.order_reviews.rename(columns={'review_id': 'review_code'}, inplace=True)
                
        self.products.drop(['product_category_name', 'product_category_name_english'], axis=1, inplace=True)

        self.order_items.drop(['order_item_id'], axis=1, inplace=True)
        self.order_items['shipping_limit_date'] = self.order_items['shipping_limit_date'].astype('datetime64')

        self.marketing_qualified_leads.fillna('no_data', inplace=True)
        self.marketing_qualified_leads = self.marketing_qualified_leads.astype({'first_contact_date': 'datetime64'})

        self.close_deals.drop(['lead_behaviour_profile', 'has_company', 'has_gtin', 'average_stock', 'declared_product_catalog_size'], axis=1, inplace=True)
        self.close_deals.fillna('no_data', inplace=True)
        self.close_deals = self.close_deals.astype({'won_date': 'datetime64'})


    def load(self):

        self.geolocation.to_sql('geolocation', engine, if_exists='append', index=False)
        self.customers.to_sql('customers', engine, if_exists='append', index=False)
        self.sellers.to_sql('sellers', engine, if_exists='append', index=False)
        self.orders.to_sql('orders', engine, if_exists='append', index=False)
        self.order_reviews.to_sql('order_reviews', engine, if_exists='append', index=False)
        self.order_payments.to_sql('order_payments', engine, if_exists='append', index=False)
        self.product_category.to_sql('product_category', engine, if_exists='append', index=False)
        self.products.to_sql('products', engine, if_exists='append', index=False)
        self.order_items.to_sql('order_items', engine, if_exists='append', index=False)
        self.marketing_qualified_leads.to_sql('marketing_qualified_leads', engine, if_exists='append', index=False)
        self.close_deals.to_sql('close_deals', engine, if_exists='append', index=False)

In [3]:
etl = Etl()

In [4]:
etl.extract()

In [5]:
etl.customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [6]:
etl.sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


In [7]:
etl.orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [8]:
etl.order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [9]:
etl.order_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [10]:
etl.product_category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [11]:
etl.products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [12]:
etl.order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [13]:
etl.marketing_qualified_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   mql_id              8000 non-null   object
 1   first_contact_date  8000 non-null   object
 2   landing_page_id     8000 non-null   object
 3   origin              7940 non-null   object
dtypes: object(4)
memory usage: 250.1+ KB


In [14]:
etl.close_deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   mql_id                         842 non-null    object 
 1   seller_id                      842 non-null    object 
 2   sdr_id                         842 non-null    object 
 3   sr_id                          842 non-null    object 
 4   won_date                       842 non-null    object 
 5   business_segment               841 non-null    object 
 6   lead_type                      836 non-null    object 
 7   lead_behaviour_profile         665 non-null    object 
 8   has_company                    63 non-null     object 
 9   has_gtin                       64 non-null     object 
 10  average_stock                  66 non-null     object 
 11  business_type                  832 non-null    object 
 12  declared_product_catalog_size  69 non-null     flo

In [5]:
add_product_category = {'product_category_name': ['pc_gamer', 'no_data', 'portateis_cozinha_e_preparadores_de_alimentos'] , 'product_category_name_english': ['pc_gamer', 'no_data','kitchen_portables_and_food_preparators']}
etl.transform(add_product_category)

In [16]:
etl.geolocation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15251 entries, 0 to 15250
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   geolocation_id   15251 non-null  int64 
 1   zip_code_prefix  15251 non-null  int64 
 2   city             15251 non-null  object
 3   state            15251 non-null  object
dtypes: int64(2), object(2)
memory usage: 476.7+ KB


In [17]:
etl.customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     99441 non-null  object
 1   geolocation_id  99441 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.3+ MB


In [18]:
etl.sellers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3095 entries, 0 to 3094
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   seller_id       3095 non-null   object
 1   geolocation_id  3095 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 72.5+ KB


In [19]:
etl.orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99441 non-null  datetime64[ns]
 5   order_delivered_carrier_date   99441 non-null  datetime64[ns]
 6   order_delivered_customer_date  99441 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [20]:
etl.order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_code              99224 non-null  object        
 1   order_id                 99224 non-null  object        
 2   review_score             99224 non-null  int64         
 3   review_creation_date     99224 non-null  datetime64[ns]
 4   review_answer_timestamp  99224 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 3.8+ MB


In [6]:
etl.order_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [21]:
etl.product_category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_id            74 non-null     int64 
 1   product_category_name          74 non-null     object
 2   product_category_name_english  74 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.9+ KB


In [22]:
etl.products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   product_id                  32951 non-null  object
 1   product_name_lenght         32951 non-null  int64 
 2   product_description_lenght  32951 non-null  int64 
 3   product_photos_qty          32951 non-null  int64 
 4   product_weight_g            32951 non-null  int64 
 5   product_length_cm           32951 non-null  int64 
 6   product_height_cm           32951 non-null  int64 
 7   product_width_cm            32951 non-null  int64 
 8   product_category_id         32951 non-null  int64 
dtypes: int64(8), object(1)
memory usage: 2.5+ MB


In [23]:
etl.order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             112650 non-null  object        
 1   product_id           112650 non-null  object        
 2   seller_id            112650 non-null  object        
 3   shipping_limit_date  112650 non-null  datetime64[ns]
 4   price                112650 non-null  float64       
 5   freight_value        112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 5.2+ MB


In [24]:
etl.marketing_qualified_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   mql_id              8000 non-null   object        
 1   first_contact_date  8000 non-null   datetime64[ns]
 2   landing_page_id     8000 non-null   object        
 3   origin              8000 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 250.1+ KB


In [25]:
etl.close_deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   mql_id                    842 non-null    object        
 1   seller_id                 842 non-null    object        
 2   sdr_id                    842 non-null    object        
 3   sr_id                     842 non-null    object        
 4   won_date                  842 non-null    datetime64[ns]
 5   business_segment          842 non-null    object        
 6   lead_type                 842 non-null    object        
 7   business_type             842 non-null    object        
 8   declared_monthly_revenue  842 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 59.3+ KB


In [ ]:
etl.load()